In [19]:
from datasets import load_dataset, DatasetDict
dataset = load_dataset('csv', data_files='/kaggle/input/2022-data/train.csv')

In [20]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import torch.nn.functional as F

# google/mt5-small
# Define the student model
student_model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")
student_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")

# Define the teacher models
teacher_model1 = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")
# teacher_model_2 = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:

# Define the loss function
def distillation_loss(student_outputs, labels, teacher_outputs_1, teacher_outputs_2, alpha_ce=0.5, alpha_distil=0.5):
    cross_entropy_loss = F.cross_entropy(student_outputs.logits.view(-1, student_outputs.logits.size(-1)), labels.view(-1))
#     if student_outputs.logits.size(-1) < teacher_outputs_1.logits.size(-1):
#     # Pad student logits with zeros (assuming teacher has more classes)
#         padding_size = teacher_outputs_1.logits.size(-1) - student_outputs.logits.size(-1)
#         student_outputs.logits = torch.nn.functional.pad(student_outputs.logits, (0, padding_size))

    distillation_loss_1 = F.kl_div(
        F.log_softmax(student_outputs.logits / 2.0, dim=-1),
        F.softmax(teacher_outputs_1.logits / 2.0, dim=-1),
        reduction='batchmean'
    )

#     distillation_loss_2 = F.kl_div(
#         F.log_softmax(student_outputs.logits / 2.0, dim=-1),
#         F.softmax(teacher_outputs_2.logits / 2.0, dim=-1),
#         reduction='batchmean'
#     )
    distillation_loss_2 =0

    distillation_loss = (distillation_loss_1 + distillation_loss_2) / 2.0

    total_loss = alpha_ce * cross_entropy_loss + alpha_distil * distillation_loss

    return total_loss

In [22]:
# def distillation_loss(student_outputs, labels, teacher_outputs_1, teacher_outputs_2=None, alpha_ce=0.5, alpha_distil=0.5):
#   """
#   Computes the combined loss for knowledge distillation.

#   Args:
#       student_outputs: A dictionary containing the student model's outputs.
#           Expected to have a key 'logits' for the logits tensor.
#       labels: A tensor containing the ground truth labels.
#       teacher_outputs_1: A dictionary containing the first teacher model's outputs.
#           Expected to have a key 'logits' for the logits tensor.
#       teacher_outputs_2 (optional): A dictionary containing the second teacher model's outputs (if using two teachers).
#           Expected to have a key 'logits' for the logits tensor. Defaults to None.
#       alpha_ce: Weight for the cross-entropy loss (default: 0.5).
#       alpha_distil: Weight for the distillation loss (default: 0.5).

#   Returns:
#       A tensor representing the combined loss.
#   """

#   cross_entropy_loss = F.cross_entropy(student_outputs.logits.view(-1, student_outputs.logits.size(-1)), labels.view(-1))

#   # Handle potential size mismatch between student and teacher logits
#   if student_outputs.logits.size(-1) < teacher_outputs_1.logits.size(-1):
#       # Pad student logits with zeros (assuming teacher has more classes)
#     padding_size = teacher_outputs_1.logits.size(-1) - student_outputs.logits.size(-1)
#     student_outputs.logits = torch.nn.functional.pad(student_outputs.logits, (0, padding_size))
    
#   softL=F.log_softmax(student_outputs.logits / 2.0, dim=-1)
#   TeachL=F.softmax(teacher_outputs_1.logits / 2.0, dim=-1)
#   if softL.size(-1) < TeachL.size(-1):
    
#     padding_size = TeachL.size(-1) - softL.size(-1)
#     softL = torch.nn.functional.pad(softL, (0, padding_size))
    
#   if TeachL.size(-1) < softL.size(-1):
    
#     padding_size =- TeachL.size(-1) +softL.size(-1)
#     TeachL = torch.nn.functional.pad(TeachL, (0, padding_size))
#   print(softL.size(-1),TeachL.size(-1))
#   print(softL.shape,TeachL.shape)

#   distillation_loss_1 = F.kl_div(
#       softL,
#       TeachL,
#       reduction='batchmean'
#   )

#   # Handle second teacher model if provided (assuming same logic for size mismatch)
#   distillation_loss_2 = 0
#   if teacher_outputs_2 is not None:
#       if student_outputs.logits.size(-1) < teacher_outputs_2.logits.size(-1):
#           padding_size = teacher_outputs_2.logits.size(-1) - student_outputs.logits.size(-1)
#           student_outputs.logits = torch.nn.functional.pad(student_outputs.logits, (0, padding_size))
#       distillation_loss_2 = F.kl_div(
#           F.log_softmax(student_outputs.logits / 2.0, dim=-1),
#           F.softmax(teacher_outputs_2.logits / 2.0, dim=-1),
#           reduction='batchmean'
#       )

#   distillation_loss = (distillation_loss_1 + distillation_loss_2) / 2.0

#   total_loss = alpha_ce * cross_entropy_loss + alpha_distil * distillation_loss

#   return total_loss


In [23]:
teacher_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
from transformers import AutoTokenizer
from datasets import load_dataset, DatasetDict
dataset = load_dataset('csv', data_files='/kaggle/input/2022-data/train.csv')

max_input_length = 100
max_target_length = 50
# Load the tokenizer for the student model
prefix = 'summarize'

# Define the encoding function
def encode_data(examples):
    inputs = [prefix + doc for doc in examples["Article"]]
    model_inputs = student_tokenizer(inputs, max_length=max_input_length, truncation=True,padding=True)

    # Setup the tokenizer for targets
    labels = student_tokenizer(text_target=examples["Summary"], max_length=max_target_length, padding=True)
#     labels = tokenizer(text_target=examples["Summary"], padding='longest', truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Define the encoding function
def encode_data2(examples):
    inputs = [prefix + doc for doc in examples["Article"]]
    model_inputs = teacher_tokenizer(inputs, max_length=max_input_length, truncation=True,padding=True)

    # Setup the tokenizer for targets
    labels = teacher_tokenizer(text_target=examples["Summary"], max_length=max_target_length, padding=True)
#     labels = tokenizer(text_target=examples["Summary"], padding='longest', truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the encoding function to the dataset
encoded_dataset = dataset['train'].map(encode_data, batched=True, remove_columns=['Article', 'Summary','Heading','id'])
encoded_dataset2 = dataset['train'].map(encode_data2, batched=True, remove_columns=['Article', 'Summary','Heading','id'])

In [25]:
encoded_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 7957
})

In [26]:
encoded_dataset2

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 7957
})

In [27]:

subset_dataset1 = encoded_dataset
# subset_size = 1000  # Specify the size of the subset you want to create

# subset_dataset1 = dataset_dict.select(range(subset_size))
subset_dataset_dict = DatasetDict({'train': subset_dataset1})
encoded_dataset=subset_dataset_dict

In [28]:

subset_dataset1 = encoded_dataset2
subset_size = 1000  # Specify the size of the subset you want to create

# subset_dataset1 = dataset_dict.select(range(subset_size))
subset_dataset_dict = DatasetDict({'train': subset_dataset1})
encoded_dataset2=subset_dataset_dict

In [29]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7957
    })
})

In [30]:
encoded_dataset = encoded_dataset2.remove_columns('token_type_ids')

In [31]:
encoded_dataset2 = encoded_dataset2.remove_columns('token_type_ids')

In [32]:
encoded_dataset2

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7957
    })
})

In [45]:
# inputs = {
#         'input_ids': batch['input_ids'],
#         'attention_mask': batch['attention_mask'],
#         'labels': batch['labels']
#     }

In [46]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq


data_collator1 = DataCollatorForSeq2Seq(tokenizer=student_tokenizer)
data_collator2 = DataCollatorForSeq2Seq(tokenizer=teacher_tokenizer)

train_dataloader1 = DataLoader(encoded_dataset['train'], batch_size=2, shuffle=True, collate_fn=data_collator1)
train_dataloader2 = DataLoader(encoded_dataset2['train'], batch_size=2, shuffle=True, collate_fn=data_collator2)


In [47]:
len(train_dataloader1)

3979

In [49]:
from torch.utils.data import DataLoader
from tqdm import tqdm  # Optional: for progress bar


optimizer = torch.optim.Adam(student_model.parameters())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# Assuming you have two pre-defined DataLoaders: train_dataloader1 and train_dataloader2
teacher_model1.to(device)
student_model.to(device)
i=0
# max_iters = min(len(train_dataloader1), len(train_dataloader2))  # Synchronize iterations
iterator1 = iter(train_dataloader1)
iterator2 = iter(train_dataloader2)
max_iters=1990
for step in tqdm(range(max_iters)):
    torch.cuda.empty_cache()
    try:
        inputs1 = next(iterator1)
        inputs2 = next(iterator2)
    except StopIteration:
        # Handle reaching the end of a DataLoader (optional, e.g., reset iterators)
        iterator1 = iter(train_dataloader1)  # Consider resetting
        iterator2 = iter(train_dataloader2)  # Consider resetting
#         inputs1 = next(iterator1).to(device)
#         inputs2 = next(iterator2).to(device)
    

    # Ensure 'input_ids' (or relevant key) is a tensor (if necessary)
    if isinstance(inputs1['input_ids'], list):
        inputs1['input_ids'] = torch.tensor(inputs1['input_ids'])  # Convert to tensor
    if isinstance(inputs2['input_ids'], list):
        inputs2['input_ids'] = torch.tensor(inputs2['input_ids'])  # Convert to tensor

    # Select teacher model (replace with your selection logic)
#     teacher_model = teacher_model_1 if step % 2 == 0 else teacher_model_2
    inputs1.to(device)
    inputs2.to(device)
    # Forward pass through student and teacher models
    student_outputs = student_model(**inputs1)
    teacher_outputs = teacher_model1(**inputs2)
#     print(student_outputs.logits.shape)
#     print(teacher_outputs.logits.shape)
    if(student_outputs.logits.shape!=teacher_outputs.logits.shape):
        i+=1
        continue
    # Compute the distillation loss
    loss = distillation_loss(student_outputs, inputs1['labels'], teacher_outputs,0)

    # Perform backpropagation
    loss.backward()

    # Update the weights
    optimizer.step()
    optimizer.zero_grad()


100%|██████████| 1990/1990 [05:04<00:00,  6.53it/s]


In [60]:
import pickle

with open('indicbart_distil.pkl','wb') as f:
   pickle.dump(student_model, f) 
    
# with open('mt5-base.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [62]:
with open('indicbart_distil_tokenizer.pkl', 'wb') as f:
    pickle.dump(student_tokenizer, f)

In [54]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [63]:
with open('/kaggle/working/indicbart_distil.pkl', 'rb') as f:
    model = pickle.load(f)

with open('/kaggle/working/indicbart_distil_tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [65]:
import pandas as pd
from rouge import Rouge
df_test=pd.read_csv("/kaggle/input/2022-data/test.csv")
df_test=df_test[:10]
def predict_summary(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=600, truncation=True)
    inputs.to("cuda")
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=40, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

df_test['predicted_summary'] = df_test['Article'].apply(predict_summary)

rouge = Rouge()


def calculate_rouge(row):
    return rouge.get_scores(row['predicted_summary'], row['Summary'], avg=True)

df_test['rouge'] = df_test.apply(calculate_rouge, axis=1)
df_test['rouge-1'] = df_test['rouge'].apply(lambda x: x['rouge-1']['f'])
df_test['rouge-2'] = df_test['rouge'].apply(lambda x: x['rouge-2']['f'])

print(df_test['rouge-1'].mean())
print(df_test['rouge-2'].mean())

0.5484455285199256
0.43516804200716497


In [67]:
print(i)

1588


In [69]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader1
)

from transformers import get_scheduler

num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

import nltk
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [71]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=774bcaa0537c245378f74309fe869035d08e6b5548cc3056e9e47afbe62fd23e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [82]:
from tqdm.auto import tqdm
import torch
import numpy as np

from datasets import load_metric

# Load ROUGE metric
rouge_metric = load_metric("rouge")

progress_bar = tqdm(range(num_training_steps))
# all_logits = []

for epoch in range(num_train_epochs):
    # Training
    model.train()
    torch.cuda.empty_cache()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        logits = outputs.logits
#         all_logits.append(logits.detach().cpu().numpy())
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
#     model.eval()
#     for step, batch in enumerate(eval_dataloader):
#         with torch.no_grad():
#             generated_tokens = accelerator.unwrap_model(model).generate(
#                 batch["input_ids"],
#                 attention_mask=batch["attention_mask"],
#             )

#             generated_tokens = accelerator.pad_across_processes(
#                 generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
#             )
#             labels = batch["labels"]

#             # If we did not pad to max length, we need to pad the labels too
#             labels = accelerator.pad_across_processes(
#                 batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
#             )

#             generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
#             labels = accelerator.gather(labels).cpu().numpy()

#             # Replace -100 in the labels as we can't decode them
#             labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#             if isinstance(generated_tokens, tuple):
#                 generated_tokens = generated_tokens[0]
#             decoded_preds = tokenizer.batch_decode(
#                 generated_tokens, skip_special_tokens=True
#             )
#             decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#             decoded_preds, decoded_labels = postprocess_text(
#                 decoded_preds, decoded_labels
#             )

#             rouge_metric.add_batch(predictions=decoded_preds, references=decoded_labels)

#     # Compute metrics
#     result = rouge_metric.compute()
#     # Extract the median ROUGE scores
#     result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
#     result = {k: round(v, 4) for k, v in result.items()}
#     print(f"Epoch {epoch}:", result)

#     # Save and upload
#     accelerator.wait_for_everyone()
#     unwrapped_model = accelerator.unwrap_model(model)
#     unwrapped_model.save_pretrained('indicbart', save_function=accelerator.save)
#     if accelerator.is_main_process:
#         tokenizer.save_pretrained('indicbart_tokeniser')
#         repo.push_to_hub(
#             commit_message=f"Training in progress epoch {epoch}", blocking=False
#         )

/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/3979 [00:00<?, ?it/s]

In [79]:
import pickle

with open('indicbart_distil_train.pkl','wb') as f:
   pickle.dump(student_model, f) 

with open('indicbart_distil_tokenizer_train.pkl', 'wb') as f:
    pickle.dump(student_tokenizer, f)

In [85]:
import pandas as pd
from rouge import Rouge
df_test=pd.read_csv("/kaggle/input/2022-data/test.csv")
# df_test=df_test[:20]
def predict_summary(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=600, truncation=True)
    inputs.to("cuda")
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=40, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

df_test['predicted_summary'] = df_test['Article'].apply(predict_summary)

rouge = Rouge()


def calculate_rouge(row):
    return rouge.get_scores(row['predicted_summary'], row['Summary'], avg=True)

df_test['rouge'] = df_test.apply(calculate_rouge, axis=1)
df_test['rouge-1'] = df_test['rouge'].apply(lambda x: x['rouge-1']['f'])
df_test['rouge-2'] = df_test['rouge'].apply(lambda x: x['rouge-2']['f'])

print(df_test['rouge-1'].mean())
print(df_test['rouge-2'].mean())

0.4747669689209107
0.34715083883702935


In [66]:

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./student_model",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

trainer = Trainer(
    model=student_model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ValueError: expected sequence of length 110 at dim 1 (got 114)

In [ ]:
# # Prepare the data
# inputs = ...  # Prepare the input data
# labels = ...  # Prepare the ground truth labels

# # Forward pass through the student model
# student_outputs = student_model(**inputs, labels=labels)

# # Forward pass through the teacher models
# with torch.no_grad():
#     teacher_outputs_1 = teacher_model_1(**inputs, labels=labels)
#     teacher_outputs_2 = teacher_model_2(**inputs, labels=labels)

# # Calculate the distillation loss
# loss = distillation_loss(student_outputs, labels, teacher_outputs_1, teacher_outputs_2)

# # Backward pass and optimization
# loss.backward()
# optimizer.step()